In [1]:
# import lettuce and modules
import lettuce as lt
# from lettuce.unit import UnitConversion

# import math, plotting, time etc. basics
import numpy as np
import torch
import time
import datetime
import matplotlib.pyplot as plt

# import specific packages
# from scipy.signal import find_peaks
import os
import psutil
import shutil

# CLASSES & METHODS
def classes and methods for prototyping

In [2]:
class EquilibriumExtrapolationOutlet(lt.AntiBounceBackOutlet):
    """Equilibrium outlet with extrapolated pressure and velocity from inside the domain
    """
    def __init__(self, lattice, direction):
        super(EquilibriumExtrapolationOutlet, self).__init__(lattice, direction)

    def __call__(self, f):
        here = [slice(None)] + self.index
        other = [slice(None)] + self.neighbor
        rho = self.lattice.rho(f)
        u = self.lattice.u(f)
        rho_w = rho[other]
        u_w = u[other]
        f[here] = self.lattice.equilibrium(rho_w[...,None], u_w[...,None])[...,0]
        return f

    def make_no_stream_mask(self, f_shape):
        no_stream_mask = torch.zeros(size=f_shape, dtype=torch.bool, device=self.lattice.device)
        no_stream_mask[[np.setdiff1d(np.arange(self.lattice.Q), self.velocities)] + self.index] = 1
        return no_stream_mask

    def make_no_collision_mask(self, grid_shape):
        no_collision_mask = torch.zeros(size=grid_shape, dtype=torch.bool, device=self.lattice.device)
        no_collision_mask[self.index] = 1
        return no_collision_mask

class ZeroGradientOutlet(object):

    def __init__(self, lattice, direction):
        # assert (isinstance(direction, list) and len(direction) in [1,2,3] and ((np.abs(sum(direction)) == 1) and (np.max(np.abs(direction)) == 1) and (1 in direction) ^ (-1 in direction))), \
        #     LettuceException("Wrong direction. Expected list of length 1, 2 or 3 with all entrys 0 except one 1 or -1, "
        #                         f"but got {type(direction)} of size {len(direction)} and entrys {direction}.")
        self.direction = np.array(direction)
        self.lattice = lattice

        #select velocities to be replaced (the ones pointing against "direction")
        self.velocities = np.concatenate(np.argwhere(np.matmul(self.lattice.stencil.e, self.direction) < -1 + 1e-6), axis=0)

        # build indices of u and f that determine the side of the domain
        self.index = []
        self.neighbor = []
        for i in self.direction:
            if i == 0:
                self.index.append(slice(None))
                self.neighbor.append(slice(None))
            if i == 1:
                self.index.append(-1)
                self.neighbor.append(-2)
            if i == -1:
                self.index.append(0)
                self.neighbor.append(1)

    def __call__(self, f):
        f[[self.velocities] + self.index] = f[[self.velocities] + self.neighbor]
        return f

    def make_no_stream_mask(self, f_shape):
        no_stream_mask = torch.zeros(size=f_shape, dtype=torch.bool, device=self.lattice.device)
        no_stream_mask[[self.velocities] + self.index] = 1
        return no_stream_mask


class NonEquilibriumExtrapolationInletU(object):
    """ Guo's boundary condition
        https://www.researchgate.net/publication/230963379_Non-equilibrium_extrapolation_method_for_velocity_and_boundary_conditions_in_the_lattice_Boltzmann_method
        and LBM book page 189
        """

    def __init__(self, lattice, units, direction, u_w):
        # assert (isinstance(direction, list) and len(direction) in [1,2,3] and ((np.abs(sum(direction)) == 1) and (np.max(np.abs(direction)) == 1) and (1 in direction) ^ (-1 in direction))), \
        #     LettuceException("Wrong direction. Expected list of length 1, 2 or 3 with all entrys 0 except one 1 or -1, "
        #                         f"but got {type(direction)} of size {len(direction)} and entrys {direction}.")
        # print("start nonEQ_init")
        self.direction = np.array(direction)
        self.lattice = lattice
        self.u_w = units.convert_velocity_to_lu(self.lattice.convert_to_tensor(u_w))

        # select velocities to be bounced (the ones pointing in "direction")
        self.velocities_out = np.concatenate(np.argwhere(np.matmul(self.lattice.stencil.e, self.direction) > 1 - 1e-6), axis=0)
        # select velocities to be replaced (the ones pointing against "direction")
        self.velocities_in = np.concatenate(np.argwhere(np.matmul(self.lattice.stencil.e, self.direction) < -1 + 1e-6), axis=0)

        # build indices of u and f that determine the side of the domain
        self.index = []
        self.neighbor = []
        for i in self.direction:
            if i == 0:
                self.index.append(slice(None))
                self.neighbor.append(slice(None))
            if i == 1:
                self.index.append(-1)
                self.neighbor.append(-2)
            if i == -1:
                self.index.append(0)
                self.neighbor.append(1)
        self.rho_old = 1.0
        if len(self.u_w.shape) > self.lattice.D:
            self.u_w = self.u_w[tuple([slice(None)] + self.index)]
        # print("u_w.shape in init NEQExtrapolInletU:", self.u_w.shape)

    def __call__(self, f):
        Tc = 100
        here = [slice(None)] + self.index
        other = [slice(None)] + self.neighbor
        # print("other in NEQEIU.call(): ", other)
        # print("here in NEQEIU.call(): ", here)
        # print("index in NEQEIU.call(): ", self.index)
        # print("f.shape:", f.shape, "f[others].shape:", f[other].shape)

        ## rho = self.lattice.convert_to_tensor(self.lattice.rho(f[other]))
        rho = self.lattice.convert_to_tensor(torch.sum(f[other], dim=0)[None,...])
        ## u = self.lattice.convert_to_tensor(self.lattice.u(f[other]))  # gibt nur die erste Ebene Knoten aus mit f[other]
        u = self.lattice.convert_to_tensor(torch.einsum("qd,q...->d...", self.lattice.e, f[other]) / rho)
        
        if self.u_w.shape == u.shape:
            u_w = self.u_w
        else:
            list = []
            for _ in u.shape: list += [1]  # pro Dimension in u_w eine 1er Liste anhängen
            # print("len(self.u_w):", len(self.u_w))
            list[0] = len(self.u_w)  # erstes Listen-Objekt wird durch die Dimensionszahl von u_w ersetzt 
            # print("self.u_w.view(list):", self.u_w.view(list))
            u_w = self.u_w.view(list).expand_as(u)
            # print("list:", list)
            # print("u_w.shape:", self.u_w.shape)
            # u_w = self.u_w.view(list)
            # print("u_w.shape:", self.u_w.shape)
            # [3, 80, 60] -> [3, 120, 80, 60]
        
        rho_self = (1 / (1 - u_w[np.argwhere(self.direction != 0).item()] 
                         * self.lattice.e[self.velocities_in[0], np.argwhere(self.direction != 0).item()]) 
                    * (torch.sum(f[[np.setdiff1d(np.arange(self.lattice.Q), [self.velocities_in, self.velocities_out])] 
                                   + self.index] + 2 * f[[self.velocities_out] + self.index], dim=0)))
        # density filtering as proposed by https://www.researchgate.net/publication/257389374_Computational_Gas_Dynamics_with_the_Lattice_Boltzmann_Method_Preconditioning_and_Boundary_Conditions
        rho_w = (rho_self + Tc * self.rho_old) / (1+Tc)
        self.rho_old = rho_w
        # print("rho_w.shape:", rho_w.shape)
        # print("u_w.shape:", u_w.shape)
        # print("f[other].shape:", f[other].shape)
        # print("rho.shape", rho.shape)
        # print("rho_self.shape:", rho_self.shape)
        # print("u.shape", u.shape)
        ## f[here] = self.lattice.equilibrium(rho_w, u_w) + (f[other] - self.lattice.equilibrium(rho, u))  ## EQLM ist anders mit torch.einsum bzw. lattice.einsum definiert... bruh
        f[here] = (torch.einsum("q,q...->q...",self.lattice.w, (rho * ((2 * torch.tensordot(self.lattice.e, u, dims=1) - torch.einsum("d...,d...->...", u, u)) / (2 * self.lattice.cs ** 2) + 0.5 * (torch.tensordot(self.lattice.e, u, dims=1) / (self.lattice.cs ** 2)) ** 2 + 1))) 
                   + (f[other] - torch.einsum("q,q...->q...", self.lattice.w, (rho * ((2 * torch.tensordot(self.lattice.e, u, dims=1) - torch.einsum("d...,d...->...", u, u)) / (2 * self.lattice.cs ** 2) + 0.5 * (torch.tensordot(self.lattice.e, u, dims=1) / (self.lattice.cs ** 2)) ** 2 + 1)))))
        return f
    
    # def make_no_stream_mask(self, f_shape):
    #     no_stream_mask = torch.zeros(size=f_shape, dtype=torch.bool, device=self.lattice.device)
    #     no_stream_mask[[self.velocities_in] + self.index] = 1
    #     return no_stream_mask
    def make_no_stream_mask(self, f_shape):
        no_stream_mask = torch.zeros(size=f_shape, dtype=torch.bool, device=self.lattice.device)
        no_stream_mask[[self.velocities_out] + self.index] = 1
        return no_stream_mask

    # def make_no_stream_mask(self, f_shape):
    #     no_stream_mask = torch.zeros(size=f_shape, dtype=torch.bool, device=self.lattice.device)
    #     no_stream_mask[[np.array(self.lattice.stencil.opposite)[self.velocities_in]] + self.index] = 1
    #     return no_stream_mask
    
    # def make_no_stream_mask(self, f_shape):
    #     no_stream_mask = torch.zeros(size=f_shape, dtype=torch.bool, device=self.lattice.device)
    #     no_stream_mask[[np.setdiff1d(np.arange(self.lattice.Q), self.velocities_in)] + self.index] = 1
    #     return no_stream_mask

# houseFlow3D by M.Kliemank, from MA-Thesis-CD-ROM "simulation_code.py"
class HouseFlow3D(object):

    def __init__(self, resolution_x, resolution_y, resolution_z,
                 reynolds_number, mach_number, lattice,
                 char_length_lu, char_length_pu,
                 char_velocity_pu, char_density_pu):
        """
            Flow class to simulate the flow around an object (mask) in 3D.
            Parameters:
            resolution_x, resolution_y, resolution_z: domain resolutions, in LU
            lattice: object of the class with the same name from lettuce
            char_length_lu: length of the base of the house, in LU
            char_length_pu: length of the base of the house, in PU
            char_velocity_pu: characteristic velocity (inlet velocity), in PU
            char_density_pu: characteristic density, in PU
        """
        self.resolution_x = resolution_x
        self.resolution_y = resolution_y
        self.resolution_z = resolution_z
        self.units = lt.UnitConversion(
            lattice,
            reynolds_number=reynolds_number,
            mach_number=mach_number,
            characteristic_length_lu=char_length_lu,
            characteristic_length_pu=char_length_pu,
            characteristic_velocity_pu=char_velocity_pu,
            characteristic_density_pu=char_density_pu,
            characteristic_density_lu=1
        )
        self._mask = np.zeros(shape=(self.resolution_x, self.resolution_y,
                                     self.resolution_z), dtype=bool)

        self.shape = (self.resolution_x, self.resolution_y, self.resolution_z)
        # self.grid = lt.RegularGrid([resolution_x, resolution_y, resolution_z],
        #                            self.units.characteristic_length_lu,
        #                            self.units.characteristic_length_pu)

    @property
    def mask(self):
        return self._mask

    @mask.setter
    def mask(self, m):
        assert isinstance(m, np.ndarray) and m.shape == self.shape
        self._mask = m.astype(bool)
        
    @property
    def grid(self):
        # THIS IS NOT USED AT THE MOMENT. QUESTION: SHOULD THIS BE ONE- OR ZERO-BASED? Indexing or "node-number"?
        xyz = tuple(self.units.convert_length_to_pu(np.linspace(0, n*self.units.characteristic_length_pu/self.units.characteristic_length_lu, n, endpoint=False)) for n in self.shape)  # tuple of lists of x,y,(z)-values/indices
        return np.meshgrid(*xyz, indexing='ij')  # meshgrid of x-, y- (und z-)values/indices

    def initial_solution(self, x):
        """Returns the initial macroscopic values (u, rho) at each lattice node
            Initialises speed using the velocity profile function and
            pressure difference from reference pressure as 0

            Inputs:
            x: grid, in LU
        """
        p = np.zeros_like(x[0], dtype=float)[None, ...]
        u = np.zeros((len(x),) + x[0].shape)
        u[0] = self.wind_speed_profile(np.where(self.mask, 0, x[2]),
                                       self.units.characteristic_velocity_pu, 0.25)
        return p, u

    @property
    def boundaries(self):
        """Returns the objects of each boundary class for use by the Simulation class
        """
        x, y, z = self.grid
        p, u = self.initial_solution(self.grid)
        return [
            lt.BounceBackBoundary(self.mask | (z < 1e-6), self.units.lattice), # quasi gleich
            ZeroGradientOutlet(self.units.lattice, [0, 0, 1]),
            lt.EquilibriumOutletP(self.units.lattice, [1, 0, 0]),
            NonEquilibriumExtrapolationInletU(self.units.lattice,
                                                 self.units, [-1, 0, 0], np.array(u))
        ]

    def house(self, o, eg_x_length, eg_y_length,
              roof_height, roof_overhang=0, angle=35):
        """Outputs mask for flow: house with square base
            and gable-roof parallel to y direction and overhang

            Inputs:
            eg_x_length: length of the house base in x-direction, in LU
            eg_y_length: length of the house base in y-direction, in LU
            roof_height: height of the top of the roof, in LU

        """
        angle = angle * np.pi / 180
        eg_height = int(round(roof_height - (eg_x_length / 2 + roof_overhang)
                              * np.tan(angle)))
        self.roof_height = roof_height
        inside_mask = np.zeros_like(self.grid[0], dtype=bool)
        inside_mask[int(o[0] - eg_x_length / 2):int(o[0] + eg_x_length / 2),
        int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2),
        :eg_height] = True
        inside_mask[int(o[0] - eg_x_length / 2 - roof_overhang):
                    int(o[0] + eg_x_length / 2 + roof_overhang),
        int(o[1] - eg_y_length / 2):
        int(o[1] + eg_y_length / 2), eg_height] = True
        inside_mask[int(o[0] - eg_x_length / 2 - roof_overhang):
                    int(o[0] + eg_x_length / 2 + roof_overhang),
        int(o[1] - eg_y_length / 2):
        int(o[1] + eg_y_length / 2), eg_height + 1:] = \
            np.where(self.units.convert_length_to_lu(
                self.grid[2][int(o[0] - eg_x_length / 2 - roof_overhang):
                               int(o[0] + eg_x_length / 2 + roof_overhang),
                int(o[1] - eg_y_length / 2):
                int(o[1] + eg_y_length / 2), eg_height + 1:]) <
                     o[2] + roof_height + 0.5 - np.tan(angle) *
                     np.abs(self.units.convert_length_to_lu(
                         self.grid[0][
                         int(o[0] - eg_x_length / 2 - roof_overhang):
                         int(o[0] + eg_x_length / 2 + roof_overhang),
                         int(o[1] - eg_y_length / 2):
                         int(o[1] + eg_y_length / 2),
                         eg_height + 1:]) - o[0]),
                     True,
                     inside_mask[int(o[0] - eg_x_length / 2 - roof_overhang):
                                 int(o[0] + eg_x_length / 2 + roof_overhang),
                     int(o[1] - eg_y_length / 2):
                     int(o[1] + eg_y_length / 2), eg_height + 1:])
        return inside_mask

    def wind_speed_profile(self, z, u_0, alpha=0.25):
        """Returns the objects of each boundary class for use by the Simulation class

            Inputs:
            z: height value or array of height values, in PU
            u_0: characteristic velocity, applies at roof_height, in PU
            alpha: profile shape exponent
        """
        return u_0 * (z / self.roof_height) ** alpha

# INPUT/OUTPUT SETTINGS

In [3]:
sim_parameters=dict()

location = 'hbrs'  # location: 'hbrs', 'beuel'
sim_parameters["name"] = "test_MK_Thesis_code_v01"

sim_parameters["output_data"] = True
sim_parameters["output_vtk"] = True
sim_parameters["vtk_fps"] = 10

if location == 'hbrs':
    sim_parameters["output_path"] = "/mnt/ScratchHDD1/Max_Scratch/lbm_simulations"  # lokal HBRS
    sim_parameters["output_path_vtk"] = sim_parameters["output_path"]
    sim_parameters["diIlio_path"] = '/home/mbille/lettuce/myTest/DiIlio_data/'  # lokal HBRS
if location == 'beuel':
    sim_parameters["output_path"] = "/home/max/Documents/lbm_simulations"  # lokal Bonn
    sim_parameters["output_path_vtk"] = sim_parameters["output_path"]
    sim_parameters["diIlio_path"] = "/home/max/lettuce/myTest/DiIlio_data/" # lokal Bonn



# SIMULATION PARAMETERS

In [4]:
sim_parameters["house_length_lu"] = 10 # RESOLUTION / char_length_lu 60
sim_parameters["house_length_pu"] = 10 # char_length_pu

sim_parameters["re"] = 20000
sim_parameters["ma"] = 0.1
sim_parameters["viscosity_pu"] = 14.852989758837 * 10**(-6)
sim_parameters["char_velocity_pu"] = sim_parameters["re"] * sim_parameters["viscosity_pu"] / sim_parameters["house_length_lu"]

sim_parameters["roof_angle"] = 15


sim_parameters["n_start"] = 0
sim_parameters["n_stop_target"] = 5000
sim_parameters["t_start"] = 0
sim_parameters["t_stop_target"] = 0

sim_parameters["device"] = "cuda"  # cuda, cpu
sim_parameters["dtype"] = "single" # single, double, half?





# TIMESTAMP & NAME

In [5]:
if sim_parameters["output_data"]:
    timestamp = datetime.datetime.now().strftime("%y%m%d_%H%M%S")
    sim_parameters["ID"] = str(timestamp) + "_" + sim_parameters["name"]
    os.makedirs(sim_parameters["output_path"]+"/"+sim_parameters["ID"])
    if sim_parameters["output_vtk"] and not os.path.exists(sim_parameters["output_path_vtk"]+"/"+sim_parameters["ID"]):
        os.makedirs(sim_parameters["output_path_vtk"]+"/"+sim_parameters["ID"])

# SIMULATOR SETUP

In [6]:
### SIMULATOR SETUP
# create objects, link/assemble

lattice = lt.Lattice(lt.D3Q27, device=torch.device("cuda"), dtype=torch.float32)
#lattice.equilibrium = lt.QuadraticEquilibrium_LessMemory(lattice)
# original MK: 360x240x180
flow = HouseFlow3D(120, 80, 60, sim_parameters["re"], mach_number=sim_parameters["ma"], lattice=lattice,
                   char_length_lu=sim_parameters["house_length_lu"], char_length_pu=sim_parameters["house_length_pu"],
                   char_density_pu=1.2250, char_velocity_pu=sim_parameters["char_velocity_pu"])

# generate house mask and set it as mask of flow
# original MK: 120, 120, 0
flow.mask = flow.house([40,  40, 0], 
                       eg_x_length=sim_parameters["house_length_lu"],
                       eg_y_length=sim_parameters["house_length_lu"],
                       roof_height=sim_parameters["house_length_lu"]*1.25,
                       # roof_overhang=6, 
                       angle=sim_parameters["roof_angle"])

collision = lt.KBCCollision3D(lattice, tau=flow.units.relaxation_parameter_lu)
streaming = lt.StandardStreaming(lattice)
simulation = lt.Simulation(flow, lattice, collision, streaming)

In [7]:
### REPORTER SETUP
# create and append reporters
vtk_reporter = lt.VTKReporter(lattice, flow,
                              interval=50,
                             # interval=int(flow.units.convert_time_to_lu(1/sim_parameters["vtk_fps"])), 
                              filename_base=sim_parameters["output_path_vtk"]+"/"+sim_parameters["ID"] +"/vtk/out")
simulation.reporters.append(vtk_reporter)
# TODO: obstacle point und cell Maske ausgeben für VTK output
# MK hat da auch den "ouput_mask" zum vtk_reporter hinzugefügt und kann das nach der Initialisierung aufrufen

# SIMULATION

In [8]:
n_steps = sim_parameters["n_stop_target"] - sim_parameters["n_start"]
t_start = time.time()

mlups = simulation.step(n_steps)

t_end = time.time()
runtime = t_end-t_start

print("MLUPS:", mlups)
print("PU-Time: ",flow.units.convert_time_to_pu(n_steps)," seconds")
print("number of steps:",n_steps)
print("runtime: ",runtime, "seconds (", round(runtime/60,2),"minutes )")

print("current GPU VRAM (MB): ", torch.cuda.memory_allocated(device="cuda:0")/1024/1024)
print("max. GPU VRAM (MB): ", torch.cuda.max_memory_allocated(device="cuda:0")/1024/1024)

[cpuLoad1,cpuLoad5,cpuLoad15] = [x / psutil.cpu_count() * 100 for x in psutil.getloadavg()]
print("CPU % avg. over last 1 min, 5 min, 15 min; ", round(cpuLoad1,2), round(cpuLoad5,2), round(cpuLoad15,2))

ram = psutil.virtual_memory()
print("current total RAM usage [MB]: " + str(round(ram.used/(1024*1024),2)) + " of " + str(round(ram.total/(1024*1024),2)) + " MB")

MLUPS: 26.567699677911936
PU-Time:  9717.745022447805  seconds
number of steps: 5000
runtime:  108.40261602401733 seconds ( 1.81 minutes )
current GPU VRAM (MB):  82.546875
max. GPU VRAM (MB):  389.138671875
CPU % avg. over last 1 min, 5 min, 15 min;  2.97 2.05 1.57
current total RAM usage [MB]: 9098.31 of 64219.95 MB
